In [ ]:
from time import time
start = time()
from IPython.display import clear_output
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y
!pip install /kaggle/input/kerasapplications -q
!pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps
clear_output()

In [ ]:
%cd /kaggle/
import os
import re
import ast
import sys
import json
import torch
import shutil
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from numba import cuda 
import pydicom as dicom
import tensorflow as tf
from itertools import chain
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import efficientnet.tfkeras as efn
sys.path.append("/kaggle/input/testutils")
from test_utils import  Test, Generator
from sklearn.model_selection import StratifiedKFold
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
sys.path.insert(0, "input/weightedboxesfusion/")
from ensemble_boxes import *

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
sub_study = sample_submission[sample_submission['id'].str.contains('_study')].reset_index(drop=True)
sub_image = sample_submission[sample_submission['id'].str.contains('_image')].reset_index(drop=True)
labels = ["negative", "typical", "indeterminate", "atypical"]
if sample_submission.shape[0]==2477:
    TRAIN_MODE = True
else:
    TRAIN_MODE = False
CLASSIFICATION = True
CROP = False
NORM = True
TRAIN_MODE

In [ ]:
TEST_PATH = '/kaggle/input/siim-covid19-detection/test/'
TEST_JPG_PATH = '/kaggle/working/test_imgs/'
CACHE_CLASS_PATH = '/kaggle/working/test_imgs_class/'
CROP_MODEL_PATH = '/kaggle/input/cropmodelb0/EffB0_3.h5' if CROP else None
MODEL_PATH = ['/kaggle/input/siim-covid19-efnb7-train-study/']
RESIZED_YOLO_PATH = '/kaggle/working/test_imgs_yolo/'
MODEL_2CL_PATH = None

In [ ]:
test_df = Test.prepare_data(
    data_path=TEST_PATH,
    path_to_save=TEST_JPG_PATH,
    train_mode=TRAIN_MODE,
    n_train_sample=20,
    norm=NORM,
    img_size=600,
    crop_model_path=CROP_MODEL_PATH,
    classification_img_path=CACHE_CLASS_PATH,
    resized_yolo_path = RESIZED_YOLO_PATH,
    yolo_img_size=1024,
    npy_yolo=True
)
test_df.head()

In [ ]:
tf.keras.backend.clear_session()
if CLASSIFICATION:
    gen = Generator(
        df=test_df,
        img_size=600,
        batch_size=1,
        norm=NORM,
        crop=CROP,
        cache_path=CACHE_CLASS_PATH
    )
    pred_df = Test.make_predictions(
        test_df_=test_df,
        model_path=MODEL_PATH,
        generator=gen,
        neg_cl=False
    )
    pred_df = pred_df.merge(test_df[['image', 'StudyInstanceUID']], on='image')

In [ ]:
def make_study_submission(pred_df_, test_df_, sub_study_, labels_, train_mod, max_id=True):
    sub_study_["StudyInstanceUID"] = sub_study_["id"].str[:-6]
    if train_mod:
        list_study_ids = pred_df_["StudyInstanceUID"].unique().tolist()
        sub_study_ = sub_study_.query("StudyInstanceUID in @list_study_ids")
    for ix in sub_study_.index.tolist():
        study_name = sub_study_.loc[ix, "StudyInstanceUID"]
        new_df = pred_df_[pred_df_["StudyInstanceUID"] == study_name]
        list_max_pred = []
        list_pred = []
        for col in labels_:
            list_max_pred.append(
                sorted(new_df[col].values, reverse=True)[
                    : (len(new_df[col].values) // 2) + 1
                ]
            )
            list_pred.append(sorted(new_df[col].values, reverse=True))
        ix_max = np.argmax([np.mean(x) for x in list_max_pred])
        if max_id:
            predictions = []
            for i in range(4):
                if i == ix_max:
                    predictions.append(
                        labels_[i] + " " + str(np.mean(list_max_pred[i])) + " 0 0 1 1"
                    )
                else:
                    predictions.append(
                        labels_[i] + " " + str(np.mean(list_pred[i])) + " 0 0 1 1"
                    )
        else:
            predictions = [
                labels_[i] + " " + str(np.mean(list_pred[i])) + " 0 0 1 1"
                for i in range(4)
            ]
        sub_study_.loc[ix, "PredictionString"] = " ".join(predictions)
    for ix in test_df_.index.tolist():
        img_name = test_df_.loc[ix, "image"]
        new_df = pred_df_[pred_df_["image"] == img_name]
        for col in labels_:
            if max_id:
                test_df_.loc[ix, col] = np.mean(
                    sorted(new_df[col].values, reverse=True)[
                        : (len(new_df[col].values) // 2) + 1
                    ]
                )
            else:
                test_df_.loc[ix, col] = np.mean(new_df[col].values)
    return sub_study_, test_df_

In [ ]:
if CLASSIFICATION:
    sub_study, test_df = make_study_submission(
        pred_df_=pred_df,
        test_df_=test_df,
        sub_study_=sub_study,
        labels_=labels,
        max_id=False,
        train_mod=TRAIN_MODE
    )
    test_df.head()
    test_df.shape
sub_study.head()
sub_study.shape

In [ ]:
shutil.rmtree(CACHE_CLASS_PATH)

# yolo

In [ ]:
YOLO_PATH = [
    '/kaggle/input/scaledyolov4/runs/exp2_yolov4lp7_1/weights/best.pt',
    '/kaggle/input/scaledyolov4p7weights0/only_c_0_42_b.pt', 
    '/kaggle/input/scaledyolov4p7weights/only_c_0_42.pt',
    '/kaggle/input/scaledyolov4p7weights12/comp_1_42.pt',
    '/kaggle/input/scaledyolov4p7weights12/comp_2_42.pt',
    '/kaggle/input/weightsyolo45class/weights/best_5cl_yolov4p7_42.pt',
    '/kaggle/input/weightsyolo43class/best.pt'
]
TXT_PATH = [
    '/kaggle/working/yolov4_test0/',
    '/kaggle/working/yolov4_test1/',
    '/kaggle/working/yolov4_test2/',
    '/kaggle/working/yolov4_test3/',
    '/kaggle/working/yolov4_test4/',
    '/kaggle/working/yolov4_test5/',
    '/kaggle/working/yolov4_test6/'
]
YOLO_CONF = 0.0001


In [ ]:
device = cuda.get_current_device()
device.reset()
!cp -r /kaggle/input/mishcudazip/mish-cuda-master /tmp/
!cd /tmp/mish-cuda-master && python setup.py build install
clear_output()

1. /kaggle/input/scaledyolov4/runs/exp2_yolov4lp7_1/weights/best.pt  0.338  
2. /kaggle/input/scaledyolov4p7weights0/only_c_0_42_b.pt  14  0.343  
3. /kaggle/input/scaledyolov4p7weights/only_c_0_42.pt  15  0.336
4. /kaggle/input/scaledyolov4p7weights12/comp_1_42.pt  16  0.338  
5. /kaggle/input/scaledyolov4p7weights12/comp_2_42.pt  17  0.339  
1+2 (17)  343  
1+3 (18)  339  
1+4 (19)  342  
1+5 (20)  339  
all (21)  344  

## Yolo 3cl

In [ ]:
os.chdir('/kaggle/input/scaledyolov4')
!python detect.py --weights {YOLO_PATH[6]} \
                  --source {RESIZED_YOLO_PATH} \
                  --img-size 1024 \
                  --conf-thres {YOLO_CONF} \
                  --iou-thres 0.5 \
                  --save-txt \
                  --output {TXT_PATH[6]} \
                  --augment

## yolo 5cl

In [ ]:
# os.chdir('/kaggle/input/scaledyolov4')
# !python detect.py --weights {YOLO_PATH[5]} \
#                   --source {RESIZED_YOLO_PATH} \
#                   --img-size 1024 \
#                   --conf-thres {YOLO_CONF} \
#                   --iou-thres 0.5 \
#                   --save-txt \
#                   --output {TXT_PATH[5]} \
#                   --augment


## Yolo 1cl

In [ ]:

# os.chdir('/kaggle/input/scaledyolov4')
# !python detect.py --weights {YOLO_PATH[0]} {YOLO_PATH[1]} {YOLO_PATH[2]} {YOLO_PATH[3]} {YOLO_PATH[4]} \
#                   --source {RESIZED_YOLO_PATH} \
#                   --img-size 1024 \
#                   --conf-thres {YOLO_CONF} \
#                   --iou-thres 0.5 \
#                   --save-txt \
#                   --output {TXT_PATH[0]} \
#                   --augment

# !python detect.py --weights {YOLO_PATH[0]} \
#                   --source {RESIZED_YOLO_PATH} \
#                   --img-size 1024 \
#                   --conf-thres {YOLO_CONF} \
#                   --iou-thres 0.5 \
#                   --save-txt \
#                   --output {TXT_PATH[0]} \
#                   --augment

# !python detect.py --weights {YOLO_PATH[1]} \
#                   --source {RESIZED_YOLO_PATH} \
#                   --img-size 1024 \
#                   --conf-thres {YOLO_CONF} \
#                   --iou-thres 0.5 \
#                   --save-txt \
#                   --output {TXT_PATH[1]} \
#                   --augment


# !python detect.py --weights {YOLO_PATH[2]} \
#                   --source {RESIZED_YOLO_PATH} \
#                   --img-size 1024 \
#                   --conf-thres {YOLO_CONF} \
#                   --iou-thres 0.5 \
#                   --save-txt \
#                   --output {TXT_PATH[2]} \
#                   --augment



# !python detect.py --weights {YOLO_PATH[3]} \
#                   --source {RESIZED_YOLO_PATH} \
#                   --img-size 1024 \
#                   --conf-thres {YOLO_CONF} \
#                   --iou-thres 0.5 \
#                   --save-txt \
#                   --output {TXT_PATH[3]} \
#                   --augment


# !python detect.py --weights {YOLO_PATH[4]} \
#                   --source {RESIZED_YOLO_PATH} \
#                   --img-size 1024 \
#                   --conf-thres {YOLO_CONF} \
#                   --iou-thres 0.5 \
#                   --save-txt \
#                   --output {TXT_PATH[4]} \
#                   --augment




In [ ]:
sub_image = sample_submission[sample_submission['id'].str.contains('_image')].reset_index(drop=True)
def bbox_preidction(
    df,
    sub_image_,
    bbox_folders,
    train_mod,
    IOU_THR=0.65,
    SKIP_BOX_THR=0.01,
    SIGMA=0.1,
    WBF=True,
    NMS=False,
    WEIGHTS=None,
    SOFT_NMS=False,
    NMW=False,
    as_is = False,
    n_cl = 1
):
    if n_cl==1 or n_cl==4:
        labels = ["opacity"]
    if n_cl==5:
        labels = ["negative", "typical", "indeterminate", "atypical", "opacity"]

    if train_mod:
        list_img_ids = df["image"].unique().tolist()
        list_img_ids = [list_img_ids[i]+'_image' for i in range(len(list_img_ids))]
        sub_image_ = sub_image_.query("id in @list_img_ids")
    for ix in sub_image_.index.tolist():
        img_name = sub_image_.loc[ix, "id"][:-6]
        w = df[df['image']==img_name]["width"].values[0]
        h = df[df['image']==img_name]["height"].values[0]
        bb_list = []
        conf_list = []
        class_list = []
        file = img_name + ".txt"
        for folder in bbox_folders:
            if file in os.listdir(folder):
                ls = np.loadtxt(folder + file)
                if ls.ndim == 1:
                    ls = ls.reshape(1, -1)
                boxes_list = ls[:, 1:5].copy()
                bl = ls[:, 1:5].copy()
                boxes_list[:, 0] = bl[:, 0] - bl[:, 2] / 2
                boxes_list[:, 1] = bl[:, 1] - bl[:, 3] / 2
                boxes_list[:, 2] = bl[:, 0] + bl[:, 2] / 2
                boxes_list[:, 3] = bl[:, 1] + bl[:, 3] / 2
                if n_cl==4:
                    class_list.append(np.zeros(ls.shape[0]))
                else:
                    class_list.append(ls[:, 0])
                bb_list.append(boxes_list)
                conf_list.append(ls[:, 5])
                
        if bb_list !=[]:
            if WBF:
                boxes, scores, classes = weighted_boxes_fusion(
                    bb_list,
                    conf_list,
                    class_list,
                    weights=WEIGHTS,
                    iou_thr=IOU_THR,
                    skip_box_thr=SKIP_BOX_THR,
                )
            elif NMS:
                boxes, scores, classes = nms(
                    bb_list, conf_list, class_list, weights=WEIGHTS, iou_thr=IOU_THR
                )
            elif SOFT_NMS:
                boxes, scores, classes = soft_nms(
                    bb_list,
                    conf_list,
                    class_list,
                    weights=WEIGHTS,
                    iou_thr=IOU_THR,
                    sigma=SIGMA,
                    thresh=SKIP_BOX_THR,
                )
            elif NMW:
                boxes, scores, classes = non_maximum_weighted(
                    bb_list,
                    conf_list,
                    class_list,
                    weights=WEIGHTS,
                    iou_thr=IOU_THR,
                    skip_box_thr=SKIP_BOX_THR,
                )
            elif as_is:
                boxes = bb_list[0]
                scores = conf_list[0]
                classes = class_list[0]
            xmin = boxes[:, 0] * w
            ymin = boxes[:, 1] * h
            xmax = boxes[:, 2] * w
            ymax = boxes[:, 3] * h
            list_pred = [
                f"{labels[int(classes[i])]} {scores[i]} {xmin[i]} {ymin[i]} {xmax[i]} {ymax[i]}"
                for i in range(len(boxes[:,0])) if labels[int(classes[i])]=='opacity' and (xmax[i] - xmin[i])<w/2
            ]
            sub_image_.loc[ix, 'PredictionString'] = " ".join(list_pred)
            if n_cl==5:
                negative = np.max(conf_list[0][labels.index('negative')])
                typical = np.max(conf_list[0][labels.index('typical')])
                indeterminate = np.max(conf_list[0][labels.index('indeterminate')])
                atypical = np.max(conf_list[0][labels.index('atypical')])
                sub_image_.loc[ix,"negative"] = negative
                sub_image_.loc[ix,"typical"] =  typical
                sub_image_.loc[ix,"indeterminate"] = indeterminate
                sub_image_.loc[ix,"atypical"] =  atypical
    return sub_image_

In [ ]:
sub_image = bbox_preidction(
    df = test_df,
    sub_image_ = sub_image,
    bbox_folders = [TXT_PATH[6]],
    IOU_THR=0.8,
    SKIP_BOX_THR=0.01,
    SIGMA=0.1,
    WBF=True,
    NMS=False,
    WEIGHTS=None,
    SOFT_NMS=False,
    NMW=False,
    train_mod = TRAIN_MODE,
    as_is=False,
    n_cl=4
)
sub_image

In [ ]:
n_cl=3
for ix in sub_image.index.tolist():
    img_name = sub_image.loc[ix, 'id'][:-6]
    if CLASSIFICATION:
        if n_cl==5:
            neg_conf = sub_image.loc[ix, 'negative']
        else:
            neg_conf = test_df[test_df['image']==img_name]['negative'].values[0]
        if 'opacity' not in sub_image.loc[ix, 'PredictionString']:
            sub_image.loc[ix, 'PredictionString'] = f'none {neg_conf} 0 0 1 1'
        if n_cl!=5:
            if neg_conf>0.5:
                sub_image.loc[ix, 'PredictionString'] = f'none {neg_conf} 0 0 1 1'
        
        raw_boxes = sub_image.loc[ix,"PredictionString"].split()
        max_score = max([float(raw_boxes[i+1]) for i in range(0,len(raw_boxes),6)])
        if max_score<0.2:
            sub_image.loc[ix, 'PredictionString'] = f'none {neg_conf} 0 0 1 1'
    else:
        if 'opacity' not in sub_image.loc[ix, 'PredictionString']:
            sub_image.loc[ix, 'PredictionString'] = 'none 1 0 0 1 1'
        else:
            raw_boxes = sub_image.loc[ix,"PredictionString"].split()
            max_score = max([float(raw_boxes[i+1]) for i in range(0,len(raw_boxes),6)])
            if max_score<0.2:
                sub_image.loc[ix, 'PredictionString'] = 'none 1 0 0 1 1'

In [ ]:
for p in os.listdir('/kaggle/working/'):
    if p!='submission.csv':
        try:
            shutil.rmtree('/kaggle/working/'+p)
        except:
            os.remove('/kaggle/working/'+p)

In [ ]:
submission = pd.concat([sub_study[['id', 'PredictionString']],sub_image[['id', 'PredictionString']]], ignore_index=True, axis=0)
submission

In [ ]:
submission.to_csv("/kaggle/working/submission.csv", index=False)
end = time()
end-start

In [ ]:
# plt.figure(figsize=(20, 10))
# sample_df = sub_image[sub_image['PredictionString']!='none 1 0 0 1 1'].sample(10).reset_index(drop=True)
# sample_df['image'] = sample_df['id'].str[:-6]
# sample_df = sample_df.merge(test_df, on='image')
# for i in range(10):
#     w = sample_df.loc[i, 'width']
#     h = sample_df.loc[i, 'height']
#     img = np.load('/kaggle/working/test_imgs_yolo/'+sample_df.loc[i,'id'][:-6]+'.npy')
#     ax = plt.subplot(2, 5, i + 1)
#     raw_boxes = sample_df.loc[i,"PredictionString"].split()
#     boxes = []
#     for i in range(0,len(raw_boxes),6):
#         boxes.append([int(1024*float(raw_boxes[i+2])/w), int(1024*float(raw_boxes[i+3])/h),int(1024*float(raw_boxes[i+4])/w),int(1024*float(raw_boxes[i+5])/h)])
#     for box in boxes:
#         rect = plt.Rectangle(
#             (box[0], box[1]),
#             (box[2] -box[0] ),
#            ( box[3] - box[1]),
#            ec="b", fc="none", lw=4.0,
#            alpha=0.3
#         )
#         ax.add_patch(rect)
#     ax.imshow(img, cmap=plt.cm.bone)
#     plt.xticks([])
#     plt.yticks([])
# plt.show();